In [ ]:
import pretty_midi
import os

def reverse_midi(input_path, output_path):
    """Create a time-reversed MIDI file."""
    midi = pretty_midi.PrettyMIDI(input_path)
    total_time = midi.get_end_time()

    reversed_midi = pretty_midi.PrettyMIDI()
    for inst in midi.instruments:
        new_inst = pretty_midi.Instrument(program=inst.program, is_drum=inst.is_drum, name=inst.name)
        for note in inst.notes:
            new_note = pretty_midi.Note(
                velocity=note.velocity,
                pitch=note.pitch,
                start=total_time - note.end,
                end=total_time - note.start
            )
            new_inst.notes.append(new_note)
        reversed_midi.instruments.append(new_inst)

    reversed_midi.write(output_path)
    print(f"✅ Reversed MIDI saved: {output_path}")

def flip_midi_pitch(input_path, output_path):
    """Flip MIDI pitches vertically (mirror over keyboard center)."""
    midi = pretty_midi.PrettyMIDI(input_path)
    pitch_min, pitch_max = 21, 108
    pitch_center = (pitch_min + pitch_max) // 2

    flipped_midi = pretty_midi.PrettyMIDI()
    for inst in midi.instruments:
        new_inst = pretty_midi.Instrument(program=inst.program, is_drum=inst.is_drum, name=inst.name)
        for note in inst.notes:
            flipped_pitch = pitch_center + (pitch_center - note.pitch)
            flipped_pitch = max(pitch_min, min(pitch_max, flipped_pitch))
            new_note = pretty_midi.Note(
                velocity=note.velocity,
                pitch=flipped_pitch,
                start=note.start,
                end=note.end
            )
            new_inst.notes.append(new_note)
        flipped_midi.instruments.append(new_inst)

    flipped_midi.write(output_path)
    print(f"✅ Flipped MIDI saved: {output_path}")

def reverse_and_flip(input_path, output_path):
    """Reverse time and flip pitch."""
    midi = pretty_midi.PrettyMIDI(input_path)
    total_time = midi.get_end_time()
    pitch_min, pitch_max = 21, 108
    pitch_center = (pitch_min + pitch_max) // 2

    transformed_midi = pretty_midi.PrettyMIDI()
    for inst in midi.instruments:
        new_inst = pretty_midi.Instrument(program=inst.program, is_drum=inst.is_drum, name=inst.name)
        for note in inst.notes:
            flipped_pitch = pitch_center + (pitch_center - note.pitch)
            flipped_pitch = max(pitch_min, min(pitch_max, flipped_pitch))
            new_note = pretty_midi.Note(
                velocity=note.velocity,
                pitch=flipped_pitch,
                start=total_time - note.end,
                end=total_time - note.start
            )
            new_inst.notes.append(new_note)
        transformed_midi.instruments.append(new_inst)

    transformed_midi.write(output_path)
    print(f"✅ Reversed + Flipped MIDI saved: {output_path}")


# 🧪 Apply all
file = "W5"
os.makedirs(f"midis/{file}", exist_ok=True)

reverse_midi(f"midis/{file}.mid", f"midis/{file}/{file}r.mid")
flip_midi_pitch(f"midis/{file}.mid", f"midis/{file}/{file}f.mid")
reverse_and_flip(f"midis/{file}.mid", f"midis/{file}/{file}rf.mid")


✅ Reversed MIDI saved: midis/W5/W5r.mid
✅ Flipped MIDI saved: midis/W5/W5f.mid
✅ Reversed + Flipped MIDI saved: midis/W5/W5rf.mid


In [6]:
import os
import subprocess
import tempfile

def midi_to_mp3_batch(midi_folder, soundfont, output_folder="mp3_output", samplerate=44100):
    os.makedirs(output_folder, exist_ok=True)

    midi_files = [f for f in os.listdir(midi_folder) if f.endswith(".mid") or f.endswith(".midi")]

    for midi_file in midi_files:
        midi_path = os.path.join(midi_folder, midi_file)
        base_name = os.path.splitext(midi_file)[0]
        mp3_path = os.path.join(output_folder, base_name + ".mp3")

        # Temporary WAV file
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as tmp_wav:
            wav_path = tmp_wav.name

        print(f"🎼 Converting {midi_file}...")

        try:
            # Fluidsynth: MIDI to WAV
            cmd_wav = [
                "fluidsynth", "-ni",
                "-F", wav_path,
                "-r", str(samplerate),
                soundfont, midi_path
            ]
            result = subprocess.run(cmd_wav, capture_output=True, text=True)
            if result.returncode != 0:
                print(f"❌ Fluidsynth error in {midi_file}:\n{result.stderr}")
                continue

            # FFmpeg: WAV to MP3
            cmd_mp3 = [
                "ffmpeg", "-y", "-i", wav_path,
                "-codec:a", "libmp3lame", "-qscale:a", "2",
                mp3_path
            ]
            result = subprocess.run(cmd_mp3, capture_output=True, text=True)
            if result.returncode != 0:
                print(f"❌ ffmpeg error in {midi_file}:\n{result.stderr}")
                continue

            print(f"✅ Saved MP3: {mp3_path}")

        finally:
            # Delete temp WAV
            if os.path.exists(wav_path):
                os.remove(wav_path)

# Example usage
midi_to_mp3_batch("frontend/public/midis/chopin", "piano.sf2", "frontend/public/mp3s/chopin")


🎼 Converting N1.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N1.mp3
🎼 Converting N10.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N10.mp3
🎼 Converting N11.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N11.mp3
🎼 Converting N12.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N12.mp3
🎼 Converting N13.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N13.mp3
🎼 Converting N14.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N14.mp3
🎼 Converting N15.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N15.mp3
🎼 Converting N16.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N16.mp3
🎼 Converting N17.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N17.mp3
🎼 Converting N18.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N18.mp3
🎼 Converting N19.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N19.mp3
🎼 Converting N2.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N2.mp3
🎼 Converting N20.mid...
✅ Saved MP3: frontend/public/mp3s/chopin\N20.mp3
🎼 Converting N3.mid...
✅ Saved MP3: frontend/public/mp3